# 項目實戰：Brazilian E-Commerce Public Dataset
Data Source：

This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil.
（資料由 Olist Store 提供，該資料集包含 2016年～2018年 在巴西多個市場下達的 10 萬個訂單的資訊）

Situation：

Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners.
（Olist 透過單一合同將巴西各地的小企業與渠道聯絡起來，商家可以透過 Olist 商店銷售自家產品，並使用 Olist 物流合作伙伴直接發貨給客戶）

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.
（客戶從 Olist 商店購買產品後，賣家會收到通知，要求他們履行該訂單。一旦客戶收到產品或預計交貨日期到期，客戶將透過電子郵件收到一份滿意度調查，並在其中提供購買體驗備註並寫下一些評論）

customer_id：key to the orders dataset. Each order has a unique customer_id.（訂單資料集的關鍵，每個訂單都有一個唯一的客戶_ID

customer_unique_id：unique identifier of a customer.（客戶的唯一識別_ID

order_id：unique identifier of an order.（訂單唯一識別_ID

order_purchase_timestamp：Shows the purchase timestamp.（購買時間戳

price：item price.（商品總和

freight_value：item freight value item (if an order has more than one item the freight value is splitted between items).（運費價格（如果訂單有多件商品，則運費價值在商品之間分割）

original_total：price + freight_value.（商品總和 + 運費

payment_value：transaction value.（客戶實際支付款項

adjustment_value：payment_value - original_total.（差額 [正數為分期付款的手續費/利息，負數為折價券或點數折抵，零為標準交易、無折扣且一次付清]

## 1. 導入數據庫

In [1]:
import numpy as np
import pandas as pd
import datetime
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import chi2_contingency

from matplotlib import rcParams
rcParams['font.sans-serif'] = ['STHeiti']

coin = pd.read_csv('data/resume data/new/olist_customers_orders_items_dataset_new.csv')
coin

,customer_id,customer_unique_id,order_id,order_purchase_timestamp,price,freight_value,original_total,payment_value,adjustment_value
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,5f79b5b0931d63f1a42989eb65b9da6e,2017-11-14 16:08:26,89.80,24.94,114.74,114.74,0.00
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,a44895d095d7e0702b6a162fa2dbeced,2017-07-16 09:40:32,54.90,12.51,67.41,67.41,0.00
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,316a104623542e4d75189bb372bc5f8d,2017-02-28 11:06:43,179.99,15.43,195.42,195.42,0.00
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,5825ce2e88d5346438686b0bba99e5ee,2017-08-16 13:09:20,149.90,29.45,179.35,179.35,0.00
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,0ab7fb08086d4af9141453c91878ed7a,2018-04-02 13:42:17,93.00,14.01,107.01,107.01,0.00
...,...,...,...,...,...,...,...,...,...
98193,fffcb937e9dd47a13f05ecb8290f4d3e,bb438f958261edabbe96fc22622499c4,620ddc9fbe3e445676a8e71d4a830821,2018-03-17 00:55:27,78.00,13.91,91.91,91.91,0.00
98194,fffecc9f79fd8c764f843e9951b11341,e5794df8573fa179a90a7b797fc4b71f,814d6a3a7c0b32b2ad929ac6328124e9,2018-03-29 16:59:26,54.90,26.46,81.36,0.64,-80.72
98195,fffeda5b6d849fbd39689bb92087f431,afbb5a642107cf6bb1ca68e863175f03,8c855550908247a7eff50281b92167a8,2018-05-22 13:36:02,47.90,15.23,63.13,63.13,0.00
98196,ffff42319e9b2d713724ae527742af25,680213db6ebd9e4f24d03280cbe10346,83b5fc912b2862c5046555ded1483ae9,2018-06-13 16:57:05,199.90,14.23,214.13,214.13,0.00


In [2]:
print('總紀錄數：', coin.count()) # 查看數據數
print('字段數量：', len(coin.columns)) # 查看欄位字段數量

coin.info()
coin.head()
coin.tail()

總紀錄數： customer_id                 98198
customer_unique_id          98198
order_id                    98198
order_purchase_timestamp    98198
price                       98198
freight_value               98198
original_total              98198
payment_value               98198
adjustment_value            98198
dtype: int64
字段數量： 9
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98198 entries, 0 to 98197
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               98198 non-null  object 
 1   customer_unique_id        98198 non-null  object 
 2   order_id                  98198 non-null  object 
 3   order_purchase_timestamp  98198 non-null  object 
 4   price                     98198 non-null  float64
 5   freight_value             98198 non-null  float64
 6   original_total            98198 non-null  float64
 7   payment_value             98198 non-null  float64
 8   adju

,customer_id,customer_unique_id,order_id,order_purchase_timestamp,price,freight_value,original_total,payment_value,adjustment_value
98193,fffcb937e9dd47a13f05ecb8290f4d3e,bb438f958261edabbe96fc22622499c4,620ddc9fbe3e445676a8e71d4a830821,2018-03-17 00:55:27,78.0,13.91,91.91,91.91,0.00
98194,fffecc9f79fd8c764f843e9951b11341,e5794df8573fa179a90a7b797fc4b71f,814d6a3a7c0b32b2ad929ac6328124e9,2018-03-29 16:59:26,54.9,26.46,81.36,0.64,-80.72
98195,fffeda5b6d849fbd39689bb92087f431,afbb5a642107cf6bb1ca68e863175f03,8c855550908247a7eff50281b92167a8,2018-05-22 13:36:02,47.9,15.23,63.13,63.13,0.00
98196,ffff42319e9b2d713724ae527742af25,680213db6ebd9e4f24d03280cbe10346,83b5fc912b2862c5046555ded1483ae9,2018-06-13 16:57:05,199.9,14.23,214.13,214.13,0.00
98197,ffffa3172527f765de70084a7e53aae8,48fd7dec70f2b104a1d5e8c5c639102b,d0e7be325a1c986babc4e1cdb91edc03,2017-09-02 11:53:32,21.8,23.70,45.50,45.50,0.00


## 2. 數據清洗

In [5]:
# 刪除無用數據（但在 SQL 階段已將 'canceled'、'unavailable' 的數據拿掉了

# 檢查缺失值
coin.isna().sum() # 查看後：沒有缺失值

# 檢查重複值
coin.duplicated().sum() # 查看後：沒有重複值

# 數據轉換（視數據內容進行轉換
# house_sales['price'] = house_sales['price'].str.replace('萬', '').astype('float')

# 數據轉換（購買時間
coin['order_purchase_timestamp'] = pd.to_datetime(coin['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')

# 異常值處理
coin[(coin.price<0) | (coin.freight_value<0) | (coin.original_total<0) | (coin.payment_value<0)] # 沒有
# 去掉異常值（因為沒有，所以沒有資料被去除
# coin = coin[(coin.area>20) & coin.area<600]
print(len(coin))

98198


In [7]:
# 商品總和 的異常值處理 IQR
Q1_p = coin['price'].quantile(0.25)
Q3_p = coin['price'].quantile(0.75)
IQR_p = Q3_p - Q1_p
low_price_p = Q1_p - 1.5*IQR_p
high_price_p = Q3_p + 1.5*IQR_p

coin[(coin.price<low_price_p) | (coin.price>high_price_p)]
coin_ex_p = coin[(coin.price>low_price_p) & (coin.price<high_price_p)] # 比 coin 少了將近八千筆資料（7581
print(len(coin_ex_p))

90346


In [9]:
# 運費價格 的異常值處理 IQR
Q1_f = coin_ex_p['freight_value'].quantile(0.25)
Q3_f = coin_ex_p['freight_value'].quantile(0.75)
IQR_f = Q3_f - Q1_f
low_freight_value_f = Q1_f - 1.5*IQR_f
high_freight_value_f = Q3_f + 1.5*IQR_f

coin_ex_p[(coin_ex_p.freight_value<low_freight_value_f) | (coin_ex_p.freight_value>high_freight_value_f)]
coin_ex_pf = coin_ex_p[(coin_ex_p.freight_value>low_freight_value_f) & (coin_ex_p.freight_value<high_freight_value_f)] # 少了將近八千筆資料（8388
print(len(coin_ex_pf))

81944


In [11]:
# 商品總和+運費價格 的異常值處理 IQR
Q1_o = coin_ex_pf['original_total'].quantile(0.25)
Q3_o = coin_ex_pf['original_total'].quantile(0.75)
IQR_o = Q3_o - Q1_o
low_original_total_o = Q1_o - 1.5*IQR_o
high_original_total_o = Q3_o + 1.5*IQR_o

coin_ex_pf[(coin_ex_pf.original_total<low_original_total_o) | (coin_ex_pf.original_total>high_original_total_o)]
coin_ex_pfo = coin_ex_pf[(coin_ex_pf.original_total>low_original_total_o) & (coin_ex_pf.original_total<high_original_total_o)] # 少了將近兩千筆資料（2237
print(len(coin_ex_pfo))

79707


In [13]:
# 客戶實際支付款項 的異常值處理 IQR
Q1_pv = coin_ex_pfo['payment_value'].quantile(0.25)
Q3_pv = coin_ex_pfo['payment_value'].quantile(0.75)
IQR_pv = Q3_pv - Q1_pv
low_payment_value_pv = Q1_pv - 1.5*IQR_pv
high_payment_value_pv = Q3_pv + 1.5*IQR_pv

coin_ex_pfo[(coin_ex_pfo.payment_value<low_payment_value_pv) | (coin_ex_pfo.payment_value>high_payment_value_pv)]
coin_ex_pfo_pv = coin_ex_pfo[(coin_ex_pfo.payment_value>low_payment_value_pv) & (coin_ex_pfo.payment_value<high_payment_value_pv)] # 少了將近六百筆資料（610
print(len(coin_ex_pfo_pv))

coin.info()
coin_ex_pfo_pv.info()

79097
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98198 entries, 0 to 98197
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   customer_id               98198 non-null  object        
 1   customer_unique_id        98198 non-null  object        
 2   order_id                  98198 non-null  object        
 3   order_purchase_timestamp  98198 non-null  datetime64[ns]
 4   price                     98198 non-null  float64       
 5   freight_value             98198 non-null  float64       
 6   original_total            98198 non-null  float64       
 7   payment_value             98198 non-null  float64       
 8   adjustment_value          98198 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 6.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 79097 entries, 0 to 98197
Data columns (total 9 columns):
 #   Column                   

## 3. 新的特徵數據構建

In [17]:
# 差價 分段（因為是以 <0、>0、==0 進行分箱，不是使用「遞增的數值邊界」，所以不能用 pd.cut，要用 np.select
# 定義條件
conditions = [
    (coin.adjustment_value < 0),
    (coin.adjustment_value > 0),
    (coin.adjustment_value == 0)
]

conditions_ex_pfo_pv = [
    (coin_ex_pfo_pv.adjustment_value < 0),
    (coin_ex_pfo_pv.adjustment_value > 0),
    (coin_ex_pfo_pv.adjustment_value == 0)
]

# 定義對應的標籤
choices = [
    'Installment payment fees/interest', 
    'Discount coupons or points redemption', 
    'Standard transaction'
]

# 執行分類（.loc 以確保在 coin_ex_pfo_pv 操作
coin.loc[:, 'adjustment_value_labels'] = np.select(conditions, choices, default='Unknown')
coin_ex_pfo_pv.loc[:, 'adjustment_value_labels'] = np.select(conditions_ex_pfo_pv, choices, default='Unknown')

coin
coin_ex_pfo_pv

,customer_id,customer_unique_id,order_id,order_purchase_timestamp,price,freight_value,original_total,payment_value,adjustment_value,adjustment_value_labels
0,00012a2ce6f8dcda20d059ce98491703,248ffe10d632bebe4f7267f1f44844c9,5f79b5b0931d63f1a42989eb65b9da6e,2017-11-14 16:08:26,89.80,24.94,114.74,114.74,0.00,Standard transaction
1,000161a058600d5901f007fab4c27140,b0015e09bb4b6e47c52844fab5fb6638,a44895d095d7e0702b6a162fa2dbeced,2017-07-16 09:40:32,54.90,12.51,67.41,67.41,0.00,Standard transaction
2,0001fd6190edaaf884bcaf3d49edf079,94b11d37cd61cb2994a194d11f89682b,316a104623542e4d75189bb372bc5f8d,2017-02-28 11:06:43,179.99,15.43,195.42,195.42,0.00,Standard transaction
3,0002414f95344307404f0ace7a26f1d5,4893ad4ea28b2c5b3ddf4e82e79db9e6,5825ce2e88d5346438686b0bba99e5ee,2017-08-16 13:09:20,149.90,29.45,179.35,179.35,0.00,Standard transaction
4,000379cdec625522490c315e70c7a9fb,0b83f73b19c2019e182fd552c048a22c,0ab7fb08086d4af9141453c91878ed7a,2018-04-02 13:42:17,93.00,14.01,107.01,107.01,0.00,Standard transaction
...,...,...,...,...,...,...,...,...,...,...
98193,fffcb937e9dd47a13f05ecb8290f4d3e,bb438f958261edabbe96fc22622499c4,620ddc9fbe3e445676a8e71d4a830821,2018-03-17 00:55:27,78.00,13.91,91.91,91.91,0.00,Standard transaction
98194,fffecc9f79fd8c764f843e9951b11341,e5794df8573fa179a90a7b797fc4b71f,814d6a3a7c0b32b2ad929ac6328124e9,2018-03-29 16:59:26,54.90,26.46,81.36,0.64,-80.72,Installment payment fees/interest
98195,fffeda5b6d849fbd39689bb92087f431,afbb5a642107cf6bb1ca68e863175f03,8c855550908247a7eff50281b92167a8,2018-05-22 13:36:02,47.90,15.23,63.13,63.13,0.00,Standard transaction
98196,ffff42319e9b2d713724ae527742af25,680213db6ebd9e4f24d03280cbe10346,83b5fc912b2862c5046555ded1483ae9,2018-06-13 16:57:05,199.90,14.23,214.13,214.13,0.00,Standard transaction


In [19]:
# 設定計算基準日 (最後一筆訂單時間 + 1天)
# pd.Timedelta 可以替換為 timedelta（結果一樣
reference_date = coin['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
reference_date_ex_pfo_pv = coin_ex_pfo_pv['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

print(reference_date)
print(reference_date_ex_pfo_pv)

2018-09-04 09:06:57
2018-09-04 09:06:57


## 4. RFM 模型分析、評比
RFM模型 的三大核心指標：
- Recency（最近一次消費）：顧客距離上次消費的時間。越近代表客戶越活躍、再次購買的可能性越高 ➡️客戶活躍度
- Frequency（消費頻率）：顧客在一段期間內購買的次數。次數越多代表客戶對品牌的忠誠度越高 ➡️客戶忠誠度
- Monetary（消費金額）：顧客在一段期間內的累積消費金額。金額越高代表客戶對公司的業績貢獻越大 ➡️客戶對產品的認可度

In [21]:
# 定義 RFM 計算函數，方便對 coin 和 coin_ex_pfo_pv 進行重複操作
def calculate_rfm(df, ref_date):
    # 以 customer_unique_id（客戶的唯一識別_ID）進行分組，計算 RFM
    rfm = df.groupby('customer_unique_id').agg({
        'order_purchase_timestamp':lambda x: (ref_date - x.max()).days, # R: 距離基準日的天數
        'order_id':'nunique',                                           # F: 獨立訂單數
        'payment_value':'sum'                                           # M: 實際支付總額
    }).reset_index()

    # 重新命名欄位
    rfm.columns = ['customer_unique_id', 'Recency', 'Frequency', 'Monetary']
    return rfm

# 產生兩組 RFM DataFrame
rfm_raw = calculate_rfm(coin, reference_date)
rfm_clean = calculate_rfm(coin_ex_pfo_pv, reference_date_ex_pfo_pv)

print(f'原始資料 RFM 客戶數: {len(rfm_raw)}')
print(f'清洗後資料 RFM 客戶數: {len(rfm_clean)}')

rfm_raw
rfm_clean

原始資料 RFM 客戶數: 94982
清洗後資料 RFM 客戶數: 76809


,customer_unique_id,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19
2,0000f46a3911fa3c0805444483337064,542,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,293,1,196.89
...,...,...,...,...
76804,fffbf87b7a1a6fa8b03f081c5f51a201,250,1,167.32
76805,fffea47cd6d3cc0a88bd621562a9d061,267,1,84.58
76806,ffff371b4d645b6ecea244b27531430a,573,1,112.46
76807,ffff5962728ec6157033ef9805bacc48,124,1,133.69


In [23]:
# 定義 RFM 分數轉換機制（1-5分）
# 使用 pandas.qcut() 按百分位數（Percentiles）切割
def score_rfm(rfm_df):
    df = rfm_df.copy() # 建立副本以避免 SettingWithCopyWarning
    
    # R 越小 (越近期)，標籤反轉 5 -> 1
    df['R_Score'] = pd.qcut(df['Recency'], q=5, labels=[5, 4, 3, 2, 1])
    # F 處理重複邊界值問題（因為這份資料超過 90% 的顧客只買過一次，使用 qcut 會因為「邊界值重複（太多 F 為 1）」而報錯
    # 使用 rank(method='first') 強制排序
    df['F_Score'] = pd.qcut(df['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
    # M 越大越好，標籤 1 -> 5
    df['M_Score'] = pd.qcut(df['Monetary'], q=5, labels=[1, 2, 3, 4, 5])

    # 轉為字串以利後續組合
    for col in ['R_Score', 'F_Score', 'M_Score']:
        df[col] = df[col].astype(str)

    # 組合 RFM 總分
    df['RFM_Segment'] = df['R_Score'] + df['F_Score'] + df['M_Score']
    return df

rfm_raw = score_rfm(rfm_raw)
rfm_clean = score_rfm(rfm_clean)

RFM 模型常見八種顧客分群：
1. 重要價值客（R↑, F↑, M↑）
這類顧客對品牌來說是價值最高的顧客！除了具有極高的活躍度與忠誠度，更是一群樂意在品牌花錢的高消費族群
（鼓勵投入資源經營重要價值客，以豐富的行銷活動給予消費誘因，容易以更低成本，帶動更高的整體營收！
2. 一般價值客（R↑, F↑, M↓）
這類型顧客貢獻度不高，但品牌忠誠度極高
（可以善用這些會員的忠誠度和認同感來創造品牌口碑，針對這群顧客提供特殊優惠來刺激消費，帶動顧客自主宣傳，有效擴大品牌聲量
3. 重要保持客（R↓, F↑, M↑）
此類型的顧客可能曾經是重要價值客。他們對於品牌的認可度高，卻已許久未消費
（建議可以對這群人發送關心信、定期 LINE 推播宣傳品牌最新消息，吸引回流購買
4. 一般保持客（R↓, F↑, M↓）
雖然這種類型的顧客品牌忠誠度高，不過最近可能消費需求改變而未回流
（可以向其宣傳品牌最新活動，增加這類型顧客的購買意願
5. 重要發展客（R↑, F↓, M↑）
需重點經營！！！
（為提升這類客群購買頻率，運用 LINE 推播刷存在感，並採取客製化的行銷活動與獎勵策略，激發其需求意願，保持他們長期關注品牌
6. 一般發展客（R↑, F↓, M↓）
這類顧客可能是第一次消費，尚處於品牌認知階段
（建議針對這群顧客規劃長期的品牌行銷活動，以培養品牌意識與認同感
7. 重要挽留客（R↓, F↓, M↑）
這類顧客有足夠的消費力且有潛力為品牌帶來高價值，需要主動關心了解許久未光顧的原因
（透過活動設計與專屬互動式訊息，創造品牌記憶點，加深品牌印象、培養信心
8. 一般挽留客（R↓, F↓, M↓）
這群顧客可能只是偶然消費一次，對品牌並不熟悉
（可以定期提供小額優惠券給他們，不需要花過多資源經營

In [25]:
# 創建 125 種組合底稿
scores = ['1', '2', '3', '4', '5']
all_combinations = list(itertools.product(scores, scores, scores))
df_map = pd.DataFrame(all_combinations, columns=['R_Score', 'F_Score', 'M_Score'])
df_map['RFM_Segment'] = df_map['R_Score'] + df_map['F_Score'] + df_map['M_Score']

In [27]:
# 商業邏輯貼標函數
def segment_customer(df):
    if df['RFM_Segment'] == '555':
        return 'Champions (冠軍客群/頂級VIP)'
    elif df['R_Score'] in ['4', '5'] and df['F_Score'] in ['4', '5'] and df['M_Score'] in ['1', '2']:
        return 'Bougie Loyal Customers (小資忠誠客群)'
    elif df['R_Score'] in ['1', '2'] and df['F_Score'] in ['4', '5'] and df[('M_Score')] in ['4', '5']:
        return 'Loyal Customers (忠誠客群)'
    elif df['R_Score'] in ['1', '2'] and df['F_Score'] in ['4', '5'] and df[('M_Score')] in ['1', '2']:
        return 'Regular Retain(一般保持客)'
    elif df['R_Score'] in ['4', '5'] and df['F_Score'] in ['1', '2'] and df['M_Score'] in ['4', '5']:
        return 'Recent & Promising (潛力新客)'
    elif df['R_Score'] in ['1', '2'] and df['F_Score'] in ['1', '2'] and df['M_Score'] in ['4', '5']:
        return 'At Risk / Can\'t Lose Them (高價值流失危機客)'
    elif df['R_Score'] in ['4', '5'] and df['F_Score'] in ['1', '2'] and df['M_Score'] in ['1', '2']:
        return 'Regularly New (一般發展客)'
    else:
        return 'Regularly Retained (一般挽留客)'

df_map['Customer_Type'] = df_map.apply(segment_customer, axis=1)

In [29]:
# 從表中的 RFM_Segment 獲取 df_map 裡的 Customer_Type（類似對答案
rfm_raw_final = pd.merge(rfm_raw, df_map[['RFM_Segment', 'Customer_Type']], 
    on=['RFM_Segment'], 
    how='left'
)

rfm_clean_final = pd.merge(rfm_clean, df_map[['RFM_Segment', 'Customer_Type']], 
    on=['RFM_Segment'], 
    how='left'
)

display(rfm_raw_final['Customer_Type'].value_counts())
display(rfm_clean_final['Customer_Type'].value_counts())

Customer_Type
Regularly Retained (一般挽留客)              58325
Recent & Promising (潛力新客)                6118
Loyal Customers (忠誠客群)                   6053
Regular Retain(一般保持客)                    6052
Regularly New (一般發展客)                    5950
At Risk / Can't Lose Them (高價值流失危機客)     5806
Bougie Loyal Customers (小資忠誠客群)          5650
Champions (冠軍客群/頂級VIP)                   1028
Name: count, dtype: int64

Customer_Type
Regularly Retained (一般挽留客)              47084
Regularly New (一般發展客)                    4968
Loyal Customers (忠誠客群)                   4920
Recent & Promising (潛力新客)                4859
Bougie Loyal Customers (小資忠誠客群)          4747
Regular Retain(一般保持客)                    4732
At Risk / Can't Lose Them (高價值流失危機客)     4708
Champions (冠軍客群/頂級VIP)                    791
Name: count, dtype: int64

In [30]:
df_map.head()
df_map['Customer_Type'].value_counts() # 計算 組合底稿 中符合相應客群的 RFM 分數個數
df_map

,R_Score,F_Score,M_Score,RFM_Segment,Customer_Type
0,1,1,1,111,Regularly Retained (一般挽留客)
1,1,1,2,112,Regularly Retained (一般挽留客)
2,1,1,3,113,Regularly Retained (一般挽留客)
3,1,1,4,114,At Risk / Can't Lose Them (高價值流失危機客)
4,1,1,5,115,At Risk / Can't Lose Them (高價值流失危機客)
...,...,...,...,...,...
120,5,5,1,551,Bougie Loyal Customers (小資忠誠客群)
121,5,5,2,552,Bougie Loyal Customers (小資忠誠客群)
122,5,5,3,553,Regularly Retained (一般挽留客)
123,5,5,4,554,Regularly Retained (一般挽留客)


In [31]:
rfm_raw
rfm_clean

,customer_unique_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90,4,1,4,414
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19,4,1,1,411
2,0000f46a3911fa3c0805444483337064,542,1,86.22,1,1,3,113
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62,2,1,1,211
4,0004aac84e0df4da2b147fca70cf8255,293,1,196.89,2,1,5,215
...,...,...,...,...,...,...,...,...
76804,fffbf87b7a1a6fa8b03f081c5f51a201,250,1,167.32,3,5,5,355
76805,fffea47cd6d3cc0a88bd621562a9d061,267,1,84.58,3,5,3,353
76806,ffff371b4d645b6ecea244b27531430a,573,1,112.46,1,5,4,154
76807,ffff5962728ec6157033ef9805bacc48,124,1,133.69,4,5,4,454


In [32]:
rfm_raw_final
rfm_clean_final

,customer_unique_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,Customer_Type
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,141.90,4,1,4,414,Recent & Promising (潛力新客)
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,27.19,4,1,1,411,Regularly New (一般發展客)
2,0000f46a3911fa3c0805444483337064,542,1,86.22,1,1,3,113,Regularly Retained (一般挽留客)
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,43.62,2,1,1,211,Regularly Retained (一般挽留客)
4,0004aac84e0df4da2b147fca70cf8255,293,1,196.89,2,1,5,215,At Risk / Can't Lose Them (高價值流失危機客)
...,...,...,...,...,...,...,...,...,...
76804,fffbf87b7a1a6fa8b03f081c5f51a201,250,1,167.32,3,5,5,355,Regularly Retained (一般挽留客)
76805,fffea47cd6d3cc0a88bd621562a9d061,267,1,84.58,3,5,3,353,Regularly Retained (一般挽留客)
76806,ffff371b4d645b6ecea244b27531430a,573,1,112.46,1,5,4,154,Loyal Customers (忠誠客群)
76807,ffff5962728ec6157033ef9805bacc48,124,1,133.69,4,5,4,454,Regularly Retained (一般挽留客)


In [33]:
# 將資料匯出，準備進入 Tableau / Power BI
df_map.to_csv('olist_df_map.csv', index=False)
rfm_raw_final.to_csv('olist_rfm_raw_final.csv', index=False)
rfm_clean_final.to_csv('olist_rfm_clean_final.csv', index=False)